In this project, we will use an RNN architecture to build a Machine Translation model.

It will use as a corpus wikipedia dumps.

Either the source or the target will be English. We will, in our case, try English to French Translation.

In [ ]:
#imports
!pip3 install numpy
!pip3 install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cu117
#or any nightly version so long as pytorch > 1.11 https://pytorch.org/
!pip3 install nltk gensim transformers d2l==1.0.0a1.post0

#In pytorch functional.py, change PILLOW_VERSION to __version__
#there are two places to change

Dataset

In [1]:
#Test samples location and preprocessing
import os
import torch
from d2l import torch as d2l

class MTFraEng(d2l.DataModule):  #@save
    def _download(self):
        d2l.extract(d2l.download(
            d2l.DATA_URL+'fra-eng.zip', self.root,
            '94646ad1522d915e7b0f9296181140edcf86a4f5'))
        with open(self.root + '/fra-eng/fra.txt', encoding='utf-8') as f:
            return f.read()

data = MTFraEng()
raw_text = data._download()
print(raw_text[:75])

@d2l.add_to_class(MTFraEng)  #@save
def _preprocess(self, text):
    # Replace non-breaking space with space
    text = text.replace('\u202f', ' ').replace('\xa0', ' ')
    # Insert space between words and punctuation marks
    no_space = lambda char, prev_char: char in ',.!?' and prev_char != ' '
    out = [' ' + char if i > 0 and no_space(char, text[i - 1]) else char
           for i, char in enumerate(text.lower())]
    return ''.join(out)

text = data._preprocess(raw_text)
print(text[:80])

@d2l.add_to_class(MTFraEng)  #@save
def _tokenize(self, text, max_examples=None):
    src, tgt = [], []
    for i, line in enumerate(text.split('\n')):
        if max_examples and i > max_examples: break
        parts = line.split('\t')
        if len(parts) == 2:
            # Skip empty tokens
            src.append([t for t in f'{parts[0]} <eos>'.split(' ') if t])
            tgt.append([t for t in f'{parts[1]} <eos>'.split(' ') if t])
    return src, tgt

src, tgt = data._tokenize(text)
src[:6], tgt[:6]

@d2l.add_to_class(MTFraEng)  #@save
def __init__(self, batch_size, num_steps=9, num_train=512, num_val=128):
    super(MTFraEng, self).__init__()
    self.save_hyperparameters()
    self.arrays, self.src_vocab, self.tgt_vocab = self._build_arrays(
        self._download())


@d2l.add_to_class(MTFraEng)  #@save
def _build_arrays(self, raw_text, src_vocab=None, tgt_vocab=None):
    def _build_array(sentences, vocab, is_tgt=False):
        pad_or_trim = lambda seq, t: (
            seq[:t] if len(seq) > t else seq + ['<pad>'] * (t - len(seq)))
        sentences = [pad_or_trim(s, self.num_steps) for s in sentences]
        if is_tgt:
            sentences = [['<bos>'] + s for s in sentences]
        if vocab is None:
            vocab = d2l.Vocab(sentences, min_freq=2)
        array = torch.tensor([vocab[s] for s in sentences])
        valid_len = (array != vocab['<pad>']).type(torch.int32).sum(1)
        return array, vocab, valid_len
    src, tgt = self._tokenize(self._preprocess(raw_text),
                              self.num_train + self.num_val)
    src_array, src_vocab, src_valid_len = _build_array(src, src_vocab)
    tgt_array, tgt_vocab, _ = _build_array(tgt, tgt_vocab, True)
    return ((src_array, tgt_array[:,:-1], src_valid_len, tgt_array[:,1:]),
            src_vocab, tgt_vocab)

@d2l.add_to_class(MTFraEng)  #@save
def get_dataloader(self, train):
    idx = slice(0, self.num_train) if train else slice(self.num_train, None)
    return self.get_tensorloader(self.arrays, train, idx)

data = MTFraEng(batch_size=3)
src, tgt, src_valid_len, label = next(iter(data.train_dataloader()))
print('source:', src.type(torch.int32))
print('decoder input:', tgt.type(torch.int32))
print('source len excluding pad:', src_valid_len.type(torch.int32))
print('label:', label.type(torch.int32))

@d2l.add_to_class(MTFraEng)  #@save
def build(self, src_sentences, tgt_sentences):
    raw_text = '\n'.join([src + '\t' + tgt for src, tgt in zip(
        src_sentences, tgt_sentences)])
    arrays, _, _ = self._build_arrays(
        raw_text, self.src_vocab, self.tgt_vocab)
    return arrays

src, tgt, _,  _ = data.build(['hi .'], ['salut .'])
print('source:', data.src_vocab.to_tokens(src[0].type(torch.int32)))
print('target:', data.tgt_vocab.to_tokens(tgt[0].type(torch.int32)))

Go.	Va !
Hi.	Salut !
Run!	Cours !
Run!	Courez !
Who?	Qui ?
Wow!	Ça alors !

go .	va !
hi .	salut !
run !	cours !
run !	courez !
who ?	qui ?
wow !	ça alors !
source: tensor([[ 84,   5,   2,   3,   4,   4,   4,   4,   4],
        [ 84,  61,  91,   2,   3,   4,   4,   4,   4],
        [161,   0,   3,   4,   4,   4,   4,   4,   4]], dtype=torch.int32)
decoder input: tensor([[  3, 108,   6,   2,   4,   5,   5,   5,   5],
        [  3, 105,  51,   2,   4,   5,   5,   5,   5],
        [  3,   6,   0,   4,   5,   5,   5,   5,   5]], dtype=torch.int32)
source len excluding pad: tensor([4, 5, 3], dtype=torch.int32)
label: tensor([[108,   6,   2,   4,   5,   5,   5,   5,   5],
        [105,  51,   2,   4,   5,   5,   5,   5,   5],
        [  6,   0,   4,   5,   5,   5,   5,   5,   5]], dtype=torch.int32)
source: ['hi', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
target: ['<bos>', 'salut', '.', '<eos>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


Word embedding

# We will use three different types of word embeddings:
# 1. Word2Vec
# 2. GloVe
# 3. FastText


In [4]:
"""## Word2Vec"""

import io
import os

# Python program to generate word vectors using Word2Vec
 
# importing all necessary modules
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import download
import warnings
 
warnings.filterwarnings(action = 'ignore')
 
import gensim
from gensim.models import Word2Vec

In [5]:
#keep in mind you have to launch the notebook inside the git folder to make this work (second one)
from inspect import getsourcefile
import sys
print(os.path.dirname(getsourcefile(lambda:0)))
print(sys.path[0])
print(os.path.abspath(sys.argv[0]))

C:\Users\gille\AppData\Local\Temp\ipykernel_9472
c:\Users\gille\OneDrive\Desktop\web\webtextanalysis
c:\Users\gille\miniconda3\envs\thesis\lib\site-packages\ipykernel_launcher.py


In [6]:
#We need to download the punkt package for tokenizing sentences
download("punkt")

[nltk_data] Downloading package punkt to D:\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
def tokenize(path = sys.path[0] + "\\samples\\alice.txt"):
    #  Reads ‘alice.txt’ file
    with io.open(path, 'r',encoding='utf8') as sample :
        s = sample.read()
        
        # Replaces escape character with space
        f = s.replace("\n", " ")
        
        data = []
        
        # iterate through each sentence in the file
        for i in sent_tokenize(f):
            temp = []
            
            # tokenize the sentence into words
            for j in word_tokenize(i):
                temp.append(j.lower())
        
            data.append(temp)

    return data

def test_similarity(model, word1, word2, model_name):
    print("Cosine similarity between '" + word1 + "' and '"+ word2 +"' - " + model_name + " : " + str(model.similarity(word1, word2)))
    
def Fast_similarity(model, word1, word2):
    print("Cosine similarity between '" + word1 + "' and '"+ word2 +"' - " + "FastText" + " : " + str(model.wv.similarity(word1, word2)))

embedding_data = tokenize()

In [ ]:
with open(sys.path[0] + "\\samples\\alice_tokenised.txt", 'w',encoding='utf8') as f:
    for line in embedding_data:
        f.write(" ".join(line) + " ")

In [ ]:
"""## Word2Vec"""
 
# Create CBOW model
w2v_model_cbow = gensim.models.Word2Vec(embedding_data, min_count = 1,
                              vector_size = 100, window = 5)
 
# Print results
test_similarity(w2v_model_cbow.wv, 'alice', 'wonderland', "CBOW")
     
test_similarity(w2v_model_cbow.wv, 'alice', 'machines', "CBOW")
 
# Create Skip Gram model
w2v_model_skip = gensim.models.Word2Vec(embedding_data, min_count = 1, vector_size = 100,
                                             window = 5, sg = 1)
 
# Print results
test_similarity(w2v_model_skip.wv, 'alice', 'wonderland', "SkipGram")
     
test_similarity(w2v_model_skip.wv, 'alice', 'machines', "SkipGram")

Cosine similarity between 'alice' and 'wonderland' - CBOW : 0.9727886
Cosine similarity between 'alice' and 'machines' - CBOW : 0.8968569
Cosine similarity between 'alice' and 'wonderland' - SkipGram : 0.6006849
Cosine similarity between 'alice' and 'machines' - SkipGram : 0.81720406


In [ ]:
"""## GloVe"""

# coding: utf-8
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

In [ ]:
#Only do this once
input_file = sys.path[0] + '\\models\\alice_glove.txt'
output_file = sys.path[0] + '\\models\\gensim_alice_glove.txt'
glove2word2vec(input_file, output_file)

(3390, 300)

In [ ]:
#once we have the tokenized file, we can call the glove model

####CALL FROM BASH glove_run.py

In [ ]:
# Load the model, can take a bit of time
output_file = sys.path[0] + '\\models\\gensim_alice_glove.txt'
glove_model = KeyedVectors.load_word2vec_format(output_file, binary=False)

In [ ]:

# Test the model
test_similarity(glove_model, 'alice', 'wonderland', "GloVe")
test_similarity(glove_model, 'alice', 'machines', "GloVe")

Cosine similarity between 'alice' and 'wonderland' - GloVe : -0.18298031
Cosine similarity between 'alice' and 'machines' - GloVe : -0.8092102


#FastText

In [8]:
"""## FastText"""
from gensim.models import FastText
from gensim.test.utils import common_texts  # some example sentences
from gensim.test.utils import datapath

model = FastText(vector_size=100, window=5, min_count=1)  # instantiate
model.build_vocab(corpus_iterable=embedding_data)
model.train(corpus_iterable=embedding_data, total_examples=len(embedding_data), epochs=100)  
Fast_similarity(model,'alice','wonderland')
Fast_similarity(model,'alice','machine')

Cosine similarity between 'alice' and 'wonderland' - FastText : 0.18689592
Cosine similarity between 'alice' and 'machine' - FastText : -0.12982063


RNN

In [ ]:
# Now we can create the RNN model that will translate from english to french using one of the previous embeddings

from torch import nn
import torch.nn.functional as F

class RNN(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(RNN, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [ ]:
#From this model we can create a loss function and an optimizer

def loss_function(tag_scores, gold_tags):
    loss_function = nn.NLLLoss()
    loss = loss_function(tag_scores, gold_tags)
    return loss

In [ ]:
#Now we can train the model
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = RNN(100, 128, 100, 100).to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

n_epoch = 100

for epoch in range(n_epoch):
    for sentence, tags in data:
        sentence = torch.tensor(sentence, dtype=torch.long).to(device)
        tags = torch.tensor(tags, dtype=torch.long).to(device)

        model.zero_grad()

        tag_scores = model(sentence)

        loss = loss_function(tag_scores, tags)
        loss.backward()
        optimizer.step()
        
    #here we can use the test data to evaluate the model
    
    losses = torch.zeros(len(test_data))
    
    for sentence, tags in test_data:
        sentence = torch.tensor(sentence, dtype=torch.long).to(device)
        tags = torch.tensor(tags, dtype=torch.long).to(device)

        tag_scores = model(sentence)

        loss = loss_function(tag_scores, tags)
        
        losses.append(loss.item())
        
    print("Epoch " + str(epoch) + " : " + str(losses.mean()))
    print("Std : " + str(losses.std()))
        

    print("Epoch: {}/{}.............".format(epoch, n_epoch), end=" ")
    print("Loss: {:.4f}".format(loss.item()))

To this model we can now try to add contextual embeddings

In [ ]:
#for contextual embedding we will use BERT

import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer

#we use bert and we will train it on the data
bert_model = BertModel.from_pretrained('bert-base-uncased')
#we will use the tokenizer to tokenize the sentences
sentences = ["I love machine learning", "I love coding in python"]
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]